# Лабораторная работа по теме "Работа с реакциями"

Объектами исследования в хемоинформатике являются не только молекулы, но и органические реакции. Поэтому существует ряд форматов для их хранения в удобном для машинной обработке виде [<a href='#1'>1</a>, <a href='#2'>2</a>].

В **RDKit** поддержка реакций реализована слабо, поэтому работать с реакциями будем только с помощью **CGRtools**.

In [1]:
from CGRtools.files import RDFRead, RDFWrite

In [2]:
with RDFRead('data/reaction1.rdf') as f:  # для чтения файлов реакций RDF
    r = next(f)  # извлечение по одной реакции за раз
    rs = f.read()  # для извлечения всех реакций в список

In [3]:
r  # реакции в CGRtools поддерживают отображение в jupyter

In [4]:
# Запись реакций также поддерживает python контексты.
with RDFWrite('rxn.rdf') as f:
    f.write(r)  # запись реакций в RDF


Реакции - это объекты, объединяющие в себе списки молекул и метаданные:

* **reactants** - реактанты [<a href='#3'>3</a>]
* **reagents** - реагенты [<a href='#4'>4</a>]
* **products** - продукты
* **meta** - словарь с метаданными

In [5]:
r.meta # метаданные реакции

{}

In [6]:
r.reactants  # доступ к списку реактантов

(<CGRtools.containers.molecule.MoleculeContainer at 0x1ec3a577890>,
 <CGRtools.containers.molecule.MoleculeContainer at 0x1ec49e8b4d0>)

In [7]:
r.products  # доступ к списку продуктов

(<CGRtools.containers.molecule.MoleculeContainer at 0x1ec49e88370>,
 <CGRtools.containers.molecule.MoleculeContainer at 0x1ec49e8b2f0>)

In [13]:
reactant1, reactant2, reactant3 = r.reactants # распаковка реактантов в отдельные молекулы
product = r.products[1]  # доступ к молекуле продукта по индексу

In [14]:
reactant2

In [15]:
product

Каждый атом в молекуле имеет уникальный номер. В реакции молекулы внутри списков реагентов, реактантов и продуктов должны иметь уникальные номера атомов. Но между реактантами и продуктами номера могут пересекаться. Пересекающиеся номер означают, что это одни и те же атомы. Такая нумерация атомов называется Атом-Атомным Отображением (ААО).

Каждый атом в молекуле имеет уникальный номер.  
В реакции молекулы внутри списков реагентов, реактантов и продуктов должны иметь уникальные номера атомов.  
Но между реактантами и продуктами номера могут пересекаться. Пересекающиеся номера означают что эти атомы одни и теже.  
Такая нумерация атомов называется Атом-Атомным отображением (ААО).  

In [16]:
for m in r.reactants:  # выполните и убедитесь, что номера атомов уникальны
    print(m)  # сигнатура молекулы
    for n, a in m.atoms():  # итерирование по атомам молекулы
        print(n, a.atomic_symbol)  # печать ААО и символа атома

O=C([H])C
1 O
2 C
3 C
4 H
N1([C@H]2[C@H](O)[C@@H]([C@H](O2)COP(OP(OC[C@H]3O[C@H]([C@H](O)[C@@H]3O)N4C=CCC(C(=O)N)=C4)([O-])=O)([O-])=O)O)C5=C(N=C1)C(N)=NC=N5
5 N
6 C
7 C
8 C
9 C
10 N
11 O
12 C
13 N
14 C
15 C
16 C
17 C
18 O
19 N
20 N
21 C
22 O
23 O
24 O
25 C
26 C
27 O
28 C
29 C
30 C
31 O
32 N
33 O
34 C
35 C
36 C
37 C
38 C
39 C
40 O
41 N
42 O
43 O
44 O
45 P
46 O
47 P
48 O
[H+]
49 H


In [17]:
for m in r.products:
    print(m)
    for n, a in m.atoms():  # итерирование по атомам молекулы.
        print(n, a.atomic_symbol)  # печать ААО и символа атома

C(O)C
50 C
51 C
52 O
C(N)(C=1C=[N+]([C@@H]2O[C@@H]([C@@H](O)[C@H]2O)COP(=O)(OP([O-])(OC[C@H]3O[C@H]([C@H](O)[C@@H]3O)N4C=5N=CN=C(C=5N=C4)N)=O)[O-])C=CC=1)=O
53 N
54 C
55 C
56 C
57 C
58 N
59 O
60 C
61 N
62 C
63 C
64 C
65 C
66 O
67 N
68 N
69 C
70 O
71 O
72 O
73 C
74 C
75 O
76 C
77 C
78 C
79 O
80 N
81 O
82 C
83 C
84 C
85 C
86 C
87 C
88 O
89 N
90 O
91 O
92 O
93 P
94 O
95 P
96 O


## <a name='cgr' style="color: black">КГР - Коденсированный Граф Реакции</a><a href='#contents'>**↑**</a>

Реакции можно легко превращать в Конденсированный Граф Реакции (КГР, англ. Condensed Graph of Reaction, CGR). Для этого необходимо атом-атомное отображение молекул реактантов в продукты. Атомы с одинаковыми номерами из реактантов и продуктов накладываются друг на друга с получением КГР. КГР позволяет описать химическое превращение реактантов в продукты в виде одного псевдомолекулярного графа, с обозначением образующихся и разрывающихся связей.

In [18]:
r  # рассматриваемая реакция

In [19]:
cgr = ~r  # CGR реакции

# или

r.compose()
print(cgr) # показана сигнатура CGR

OC1C(C(OC1COP(OP([O-])(=O)OCC2OC(C(O)C2O)[N+]3=CC(C(N)=O)=CC=C3)([O-])=O)N4C=NC5=C4N=CN=C5N)O.OC6C(COP(=O)([O-])OP(OCC7OC(N8C=CCC(C(=O)N)=C8)C(O)C7O)(=O)[O-])OC(C6O)N9C=NC%10=C9N=CN=C%10N.C(C)O.CC([H])=O.[H+]


In [20]:
# КГР позволяет анализировать реакционные центры. Ниже показаны номера атомов реакционных центров:
cgr.centers_list

()

## <a name='ref' style="color: black">Ссылки</a><a href='#contents'>**↑**</a>

<a name='1' style="color: black">1. </a>[Введение в хемоинформатику: Компьютерное представление химических структур: учеб. пособие / Т.И. Маджидов, И.И. Баскин, И.С. Антипин, А.А. Варнек. – Казань, Москва, Страсбург, 2020. - С. 143-155](https://edu.kpfu.ru/mod/url/view.php?id=300426).

<a name='2' style="color: black">2. </a>Введение в хемоинформатику: учеб. пособие. Ч. 5. Информатика химических реакций / И.И. Баскин, Т.И. Маджидов, А.А. Варнек. - Казань: Изд-во Казан. ун-та, 2017. - 244 с.  

<a name='3' style="color: black">3. </a>Реактанты - молекулы, вступающие в реакцию с образованием продуктов из атомов в своем составе.  

<a name='4' style="color: black">4. </a>Реагенты - молекулы, вступающие в реакцию или катализирующие ее без передачи атомов в продукты.  

# Вопросы

1. Чем схожи и чем отличаются способы работы с реакциями с помощью библиотеки CGRtools?
2. Опишите, как можно исправить ошибки атом-атомного отображения с помощью CGRtools. Приведите пример (не из тюториала).